 ### Form Submition:

 1. Navigate to you post-detail.html template
 2. Set the method of the comment form to POST
 3. The action?
    - We want to send the form to the same page but this time with a post request!
    - So use the url tag and provide it with the name of the url.
    - you should also send post.slug to the url

Your form should look like the following block:

In [ ]:
<form action='{% url "blog-post-detail" post.slug %}' method="post">

At the moment we dont have a way of handeling the PostDetailView.

4. To fix this we need to redefine our PostDetailView
    - instead of extending it from DetailView, extend it from View
    - Define post and get method
    - Delete the get_context... method
    - Manulay handle post and get

Your PostDetailView should look like the following block:

In [ ]:
from django.shortcuts import render
from .fomrs import CommentForm
from .models import Post
from django.http import HttpResponseRedirect
from django.urls import reverse

class PostDetailView(View):
    model = Post
    template_name = "blog/post-detail.html"

    def get(self, request, slug):
        post = Post.objects.get(slug = slug)
        return render(request, 'blog/post-detail.html', {
            'post':post,
            'post_tags': post.tags.all(),
            'form':CommentForm()
        })

    def post(self, request, slug):
        comment_form = CommentForm(request.POST)
        if comment_form.is_valid():
            comment_form.save()
            return HttpResponseRedirect(reverse('blog-post-detail', args=[slug]))
        
        post = Post.objects.get(slug = slug)
        return render(request, 'blog/post-detail.html', {
            'post':post,
            'post_tags': post.tags.all(),
            'form':comment_form
        })

We have a problem here:

Since we do not use the whole fomr to construct our form in the template (the post field is excluded), we can not save it the way it is and we need to modify it before saving it<br>

We can fix this by adding an extra input argument to the .save method.<br>
This extra input argument is called commit and we pass False.<br>
This tells django that instead of saving the form to the database, it has to create an object from it (pass it to a new variable).<br>
Now, we can edit it before saving it to the database.<br>
To do this you just need to pass the post to it like it is an atrribute of the new object form.<br>

Your PostDetailView should look like the following block:

In [ ]:
class PostDetailView(View):
    model = Post
    template_name = "blog/post-detail.html"

    def get(self, request, slug):
        post = Post.objects.get(slug = slug)
        return render(request, 'blog/post-detail.html', {
            'post':post,
            'post_tags': post.tags.all(),
            'form':CommentForm()
        })

    def post(self, request, slug):
        comment_form = CommentForm(request.POST)
        post = Post.objects.get(slug = slug)

        if comment_form.is_valid():
            comment = comment_form.save(commit=False)
            comment.post = post
            comment.save()
            return HttpResponseRedirect(reverse('blog-post-detail', args=[slug]))
        return render(request, 'blog/post-detail.html', {
            'post':post,
            'post_tags': post.tags.all(),
            'form':comment_form
        })

Now, everything should work just fine...<br>
The only thing that we need to do is to display the comments under each post.<br>
We will do it in the next notebook<br>


Before we move to the next notebook modify the admin.py file so that we can see all the comments in the administrative section.<br>